In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import ops
np.random.seed(1)
from PIL import Image

In [ ]:
def load_dataset():
    train_dataset = h5py.File('~/Users/fanshutong/Desktop/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('~/Users/fanshutong/Desktop/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = int(math.floor(m/mini_batch_size)) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [ ]:
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

OSError: ignored

In [ ]:
class DataGen:
  def _init_(self, X_train, X_validation, y_train, y_validation, sampleSize = 64):
    self.X_train = X_train
    self.y_train = y_train
    self.X_validation = X_validation
    self.y_validation = y_validation
    self.sampleSize = sampleSize

  def trainGen(self):
    while True:
      indx = np.random.choice(range(len(self.X_train)), self.sampleSize, replace = True) # dataset big so true
      X = self.X_train[indx]
      y = self.y_train[indx]

      yield X,y

  def validationGen(self):
    while True:
      indx = np.random.choixe(range(len(self.X_validation)), self.sampleSize, replace = True) # dataset big so true
      X = self.X_validation[indx]
      y = self.y_validation[indx]

      yield X,y

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, Sequential

from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPool2D

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
model = Sequential()

model.add(Conv2D(16, kernel_size = 3, activation = 'relu', input_shape = (64, 64, 3)))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())
model.add(Dense(6, activation = 'softmax'))

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.000001),
              loss = 'categorical_crossentropy',
              metrics= ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 64)         0

In [ ]:
weightapath = 'cnn_model_weight.model.hdf5'


In [ ]:
checkpointers = [ModelCheckpoint(filepath = weightapath,
                                 monitor = 'val_loss',
                                 verbose = 1,
                                 save_best_only = True),
                 EarlyStopping(monitor = 'val_loss',
                               min_delta = 0.00001,
                               verbose = 1,
                               patience = 200,
                               mode = 'min')]

In [ ]:
datagen = DataGen(X_train_orig, Y_train_orig, X_test_orig, Y_test_orig)

In [ ]:
model.fit(datagen.trainGen(),
          validation_data = datagen.validationGen(),
          epochs =1000,
          callbacks = checkpointers,
          verbose = True,
          steps_per_epoch = 6,
          validation_steps = 6)

In [ ]:
a = [[-1,0,1],[1,0,-1]]
a

[[-1, 0, 1], [1, 0, -1]]

In [ ]:
np.cov(a)

array([[ 1., -1.],
       [-1.,  1.]])

In [ ]:
import pandas as pd

a = pd.DataFrame(a)

In [ ]:
a.dot(a.T)

,0,1
0,2,-2
1,-2,2


In [ ]:
a

,0,1,2
0,1,2,3
1,3,2,1


In [ ]:
np.cov(a)

array([[ 1., -1.],
       [-1.,  1.]])